# Библиотеки

In [9]:
from zlib import crc32

import numpy as np
import pandas as pd
import scipy.stats as st
from statsmodels.formula.api import ols
import statsmodels.api as sm
import re

# Код генерации выборки

In [6]:
EMAIL_REGEX = re.compile(r"[^@]+@phystech.edu")

def generate_dataset(code):
    rs = np.random.RandomState(code)
    tip = rs.randint(low = 0, high = 4)
    C1 = rs.choice([1, 5], size = 1000)
    C2 = rs.choice([2, 4], size = 1000)
    if tip == 0 :
        data = rs.randn(1000)
    if tip == 1:
        data = []
        for c1,c2 in zip(C1, C2):
            if c1 == 1:
                data.append(rs.randn(1) + 1)
            if c1 == 5:
                data.append(rs.randn(1) - 1)
        data = np.array(data)
    if tip == 2:
        data = []
        for c1,c2 in zip(C1, C2):
            if c1==1:
                c1 = 0
            else:
                c1= 1
            if c2 ==2:
                c2 = 0
            else:
                c2 = 1
            data.append(rs.randn(1) + c1 + c2)
    if tip == 3:# on c1, c2
        data = []
        for c1,c2 in zip(C1,C2):
            if c1==1:
                c1 = 0
            else:
                c1= 1
            if c2 ==2:
                c2 = 0
            else:
                c2 = 1
            data.append(rs.randn(1) + c1 + c2 + 5*c1*c2)
        data = np.array(data)
    return np.hstack([np.array(data).reshape(1000, 1), C1.reshape(1000, 1), C2.reshape(1000, 1)])

# Генерация выборки для вашей почты

<span style="color:red">
    ВАЖНО!
    Почта, которую укажете ниже и почта с которой Вы отправляете домашнее задание должна совпадать!
    В момент проверки задания алгоритм генерирует выборку на основе почты из анкеты!
</span>

Внимательно проверьте почту для которой выполняется задание!

In [7]:
task = dict()
task['mail'] = input(prompt='Enter your mail: ')
assert EMAIL_REGEX.match(task['mail']), 'Not a mail in the phystech.edu domain'
task['id'] = crc32(task['mail'].encode('utf-8'))
task['data'] = generate_dataset(task['id'])

task

Enter your mail: bogdanov.ai@phystech.edu


{'mail': 'bogdanov.ai@phystech.edu',
 'id': 1162440694,
 'data': array([[ 0.97312444,  1.        ,  2.        ],
        [-2.88158553,  5.        ,  4.        ],
        [-0.54258116,  5.        ,  2.        ],
        ...,
        [ 0.11977129,  1.        ,  2.        ],
        [ 0.65151194,  1.        ,  2.        ],
        [-2.12983409,  5.        ,  4.        ]])}

# Работа с выборкой

In [11]:
data = pd.DataFrame(task['data'], columns=['f', 'C1', 'C2'])

In [13]:
lm = ols('f ~ C(C1) * C(C2)', data=data).fit()
table = sm.stats.anova_lm(lm, typ=1)
table

,df,sum_sq,mean_sq,F,PR(>F)
C(C1),1.0,1026.429911,1026.429911,1010.802597,1.088994e-153
C(C2),1.0,0.802681,0.802681,0.790461,3.741756e-01
C(C1):C(C2),1.0,0.095791,0.095791,0.094333,7.588036e-01
Residual,996.0,1011.398461,1.015460,NaN,NaN


## Проверить с использованием двухфакторного дисперсионного анализа, влияет ли переменная C1 на переменную f в данной линейной модели?

In [30]:
table.loc['C(C1)', 'PR(>F)'] < 0.05

True

## Проверить с использованием двухфакторного дисперсионного анализа, влияет ли переменная C2 на переменную f в данной линейной модели?

In [31]:
table.loc['C(C2)', 'PR(>F)'] < 0.05

False

## Проверить с использованием двухфакторного дисперсионного анализа, влияет ли межфакторное взаимодействие на переменную f в данной линейной модели?

In [33]:
table.loc['C(C1):C(C2)', 'PR(>F)'] < 0.05

False